In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import collections
import numpy as np
import gensim
from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

class TextLoader:
    def __init__(self, path):
        with open(path, "r", encoding='utf-8') as _file:
            self.text = _file.read().split()
            # self.text = self.text[:10000]

        self.song2vec = Word2Vec.load("all_word2vec_model")
        self.vocab, self.words = self.build_vocab()

        self.X = self.text[:] # 텍스트 파일 전체를 복사 - self.text 와 같은 의미
        self.y = [self.text[0]] + self.text[1:] # 

    def build_vocab(self):
        vocab_inv = list(self.song2vec.wv.vocab.keys()) # key 값을 리스트화 / 글자만
        vocab = {x: i for i, x in enumerate(vocab_inv)} # 0,1,2 등 인덱스와 단어를 dict 로 매칭시켜놓음
        return vocab, vocab_inv

    def next_batch(self, batch_size, seq_length):
        start = np.random.randint(0, len(self.X)-batch_size*seq_length) # 랜덤으로 위치를 정함 - 끝의 값을 구하면 안됨 / 시작 위치를 글자를 다 배치사이즈와 시퀀스렝스로 구함 // 마지막까지는 안가겠다는 뜻
        end   = start + batch_size*seq_length # 몇 단어를 가져올지

        X_words = self.X[start:end]# 말그대로 글자
        y_words = self.y[start:end]

        X_idx = np.empty((batch_size, seq_length), dtype=np.int64) # 글자의 인덱스
        y_idx = np.empty((batch_size, seq_length), dtype=np.int64)
        X_wv = np.empty((batch_size, seq_length, 100)) # 글자의 word2vec 
        y_wv = np.empty((batch_size, seq_length, 100))
        # 위에서 만들어준 자리에 따라 (저장공간 설정하는 과정) 아래에서 for 문을 돌며 값을 가져옴 / 그냥 하면 안되는 이유 : append는 느림, numpy의 경우에는 1 2 3 4 붙어있어야 함 / 5를 넣는다 하면 이걸 어딘가 복사해서 5를 붙여야 함.
        for i in range(batch_size):
            for j in range(seq_length):
                X_idx[i, j] = self.vocab[X_words[i*seq_length+j]]
                y_idx[i, j] = self.vocab[y_words[i*seq_length+j]]

                X_wv[i, j] = self.song2vec.wv[X_words[i*seq_length+j]]
                y_wv[i, j] = self.song2vec.wv[y_words[i*seq_length+j]]

        return X_wv, X_idx, y_wv, y_idx

In [3]:
num_layers  = 3
hidden_size = 512
batch_size  = 1 # 1글자
# max_length  = 1 # 1로 하는것으로~

loader = TextLoader("all_sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [4]:
X = tf.placeholder(tf.float32, [None, None, 50])  # 50 = word2vec size
# x_one_hot = tf.one_hot(X, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, X,
                                    initial_state=initial_state, dtype=tf.float32)


outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.linear(outputs, vocab_size) # linear
#                                 ,activation_fn=None)

In [5]:
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [22]:
# 시작 글자 생성
sentence = ["월요일", "정말", "미워"]
print(sentence)
print("Start with:", " ".join(sentence))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "./new_test_model")  # 저장된 모델 불러오기 
    
    # [배치사이즈, max_length, word2vec size]
    vec = np.empty((1, len(sentence), 50)) # 시작 글자를 주고 다음 단어를 예측 - 시작으로 준 것을 전부 다 입력으로 넣겠다
    for i, word in enumerate(sentence):
        vec[:, i, :] = loader.song2vec.wv[word]
    
    # 매 이터레이션마다 글자 하나씩 생성
    state = sess.run(states, feed_dict={X: vec}) #입력단어 sentence 이후에 들어올 단어를 예측
    for i in range(15): # for문을 돌면서 풀어헤치는 중!!
        vec = loader.song2vec.wv[sentence[-1]].reshape(1, 1, 50)
        
        pred_char, state = sess.run([pred, states], 
            feed_dict={X: vec, initial_state: state}) # 원래 initial stete 는 0 이었으나
        # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌 // 입력단어들을 그 다음 스텝에 넣어주는 것임
        
        pred_char = loader.words[pred_char[0][-1]]
        sentence.append(pred_char)

for i, word in enumerate(sentence):
    print(word, end=" ")
    if (i+1) % 5 == 0:
        print()

['월요일', '정말', '미워']
Start with: 월요일 정말 미워
INFO:tensorflow:Restoring parameters from ./new_test_model
월요일 정말 미워 망설이나요 망설이나요 
울지않겠어 울지않겠어 울지않겠어 울지않겠어 울지않겠어 
울지않겠어 울지않겠어 울지않겠어 울지않겠어 모든걸 
모든걸 투지로 투지로 